# Interactive plotting with Bokeh in GemGIS

## Set File Paths

If you downloaded the latest `GemGIS` from the Github repository, append the path so that the package can be imported successfully. In addition, the file path to the files within the `gemgis_data` folder is set. You can download the data [here](https://github.com/cgre-aachen/gemgis_data/archive/master.zip) if you have not done so.

In [1]:
import sys
sys.path.append('../../../../gemgis')
file_path ='../../../../gemgis_data/data/38_interactive_plotting_with_bokeh_in gemgis/'

## Loading Data

In [32]:
import gemgis as gg
import geopandas as gpd
import rasterio

points = gpd.read_file(file_path + 'interfaces_points.shp')
points.head()

,id,formation,geometry
0,None,Ton,POINT (19.150 293.313)
1,None,Ton,POINT (61.934 381.459)
2,None,Ton,POINT (109.358 480.946)
3,None,Ton,POINT (157.812 615.999)
4,None,Ton,POINT (191.318 719.094)


In [38]:
data = rasterio.open(file_path + 'raster.tif').read(1)

In [36]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:3857'
with rasterio.open(file_path + 'raster.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(file_path + 'raster_reproj.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
            
dataset_repr = rasterio.open(file_path + 'raster_reproj.tif')
dataset_repr

<open DatasetReader name='../../../../gemgis_data/data/38_interactive_plotting_with_bokeh_in gemgis/raster_reproj.tif' mode='r'>

In [28]:
points = gg.vector.extract_xy(gdf=points)
points.head()

,formation,geometry,X,Y
0,Ton,POINT (19.150 293.313),19.15,293.31
1,Ton,POINT (61.934 381.459),61.93,381.46
2,Ton,POINT (109.358 480.946),109.36,480.95
3,Ton,POINT (157.812 615.999),157.81,616.00
4,Ton,POINT (191.318 719.094),191.32,719.09


In [29]:
from bokeh.models import ColumnDataSource

point_source = ColumnDataSource(points.drop('geometry', axis=1))
point_source

ColumnDataSource(id='1640', ...)

In [30]:
from bokeh.models import HoverTool
my_hover = HoverTool()


IndentationError: unexpected indent (<ipython-input-30-05ab7df0ecf2>, line 2)

In [51]:
from bokeh.plotting import figure, show
from bokeh.io import show, output_notebook
output_notebook()

p = figure(title="Test",x_axis_label='x', y_axis_label='y', 
        match_aspect=True,  
        plot_width=800)
dem_renderer = p.image(image = [data], x=0, y=0, dw = 972, dh = 1069, legend_label="DEM", palette = 'RdYlGn11')
#p.circle('X', 'Y', source=point_source, color='red', size=10)

dem_tool = HoverTool(
        tooltips=[
            ('Height', '@image{0.00} m')
        ], renderers=[dem_renderer]
    )
p.add_tools(dem_tool)
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...